## AlphaNeo for Max Length of 3 (depth of 4)

```
tensorboard --logdir runs
```

In [1]:
DBG_VAR = None

In [2]:
import logging 
logging.basicConfig(level=logging.CRITICAL)

In [3]:
import os
import itertools
import copy
import random
import fcntl

from pathlib import Path

In [4]:
import tyrell.spec as S
from tyrell.interpreter import Interpreter, PostOrderInterpreter, GeneralError, InterpreterError
from tyrell.enumerator import Enumerator, SmtEnumerator, RandomEnumerator, DesignatedEnumerator, RandomEnumeratorS, ExhaustiveEnumerator
from tyrell.decider import Example, ExampleConstraintPruningDecider, ExampleDecider, TestDecider
from tyrell.synthesizer import Synthesizer
from tyrell.logger import get_logger
from sexpdata import Symbol
from tyrell import dsl as D
from typing import Callable, NamedTuple, List, Any

In [5]:
# import pickle
import dill as pickle
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from tensorboardX import SummaryWriter

use_cuda = torch.cuda.is_available()
print("use_cuda: {}".format(use_cuda))

use_cuda: False


In [6]:
# Morpheus Version
from utils_morpheus import *

In [7]:
torch.__version__

'1.0.0'

In [8]:
class ListModule(object):
    def __init__(self, module, prefix, *args):
        self.module = module
        self.prefix = prefix
        self.num_module = 0
        for new_module in args:
            self.append(new_module)
    
    def append(self, new_module):
        if not isinstance(new_module, nn.Module):
            raise ValueError('Not a Module')
        else:
            self.module.add_module(self.prefix + str(self.num_module), new_module)
            self.num_module += 1
            
    def __len__(self):
        return self.num_module
    
    def __getitem__(self, i):
        if i<0 or i>=self.num_module:
            raise IndexError('Out of bound')
        return getattr(self.module, self.prefix+str(i))

In [9]:
class AlphaNeo(nn.Module):
    def __init__(self, p_config=None):
        super(AlphaNeo, self).__init__()
        self.config = p_config
        
        self.lstm = nn.LSTMCell(
            input_size = self.config["lstm_input_size"],
            hidden_size = self.config["fcs"][0],
            bias = True,
        )
        
        self.fcs = ListModule(self, "fc_")
        for i in range(1,len(self.config["fcs"])):
            self.fcs.append(
                nn.Linear(self.config["fcs"][i-1], self.config["fcs"][i])
            )
    
    '''
    single batch behavior, no batch dim expected
    '''
    def forward(self, p1, p2, hc=None):
        # p1, p2: (1, ?)
        # hc: previous hidden state, if None, start a new sequence
        
        # [(1, ?), (1, ?), ...] -> (1, ?*k), flatten
        d_known = torch.cat([p1,p2], dim=1)
        
        if hc is None:
            hc = self.init_hidden()
        
        h_t, c_t = self.lstm(d_known, hc)
        
        tmp1 = h_t
        for i in range(len(self.fcs)-1):
            tmp1 = F.relu(self.fcs[i](tmp1))
        
        # (1, #production_rules)
        tmp1 = F.log_softmax(self.fcs[len(self.fcs)-1](tmp1), dim=1)
        
        # hidden states can be reused
        return tmp1, (h_t,c_t)
    
    def init_hidden(self):
        if use_cuda:
            return (torch.zeros(1,self.config["fcs"][0]).cuda(),
                    torch.zeros(1,self.config["fcs"][0]).cuda())
        else:
            return (torch.zeros(1,self.config["fcs"][0]),
                    torch.zeros(1,self.config["fcs"][0]))

In [10]:
class ProgramSpaceChainOneNB(object):
    # NOTICE:
    # Due to the nature of rpy2, never deep copy an interpreter
    # Interpreter is always shared
    def __init__(self, p_spec, p_interpreter, p_eq, p_input, p_output):
        self.spec = p_spec
        self.builder = D.Builder(self.spec)
        self.interpreter = p_interpreter
        self.eq = p_eq
        self.input = p_input
        self.output = p_output
        
        self.outv_list = [] # list of Example.output: [output1, output2, ....]
        self.prog_list = [] # list of Nodes
        self.sexp_list = [] # list of sexps: [Symbol(...),...]
        
        self.PARAM_NODE = self.builder._from_sexp([Symbol('@param'), 0])
        
        # generate shells (of depth 1)
        tmp_enumerator = ExhaustiveEnumerator(m_spec, max_depth=2)
        self.shell_list = [] # list of shells (prog/Nodes)
        while True:
            tmp_shell = tmp_enumerator.next()
            if tmp_shell is None:
                break
            
            if len(tmp_shell.children)>0:
                # should have at least one child
                self.shell_list.append(tmp_shell)
        # NOTICE: str(prog) is NOT sexp
        self.str_shell_list = [str(p) for p in self.shell_list]
        self.str_shell_dict = {
            self.str_shell_list[i]:i 
            for i in range(len(self.str_shell_list))
        }
        
    '''
    convert the prog_list to a full program
    (chain program is assumed)
    '''
    def get_full_prog(self, prog_list):
        prog_list = copy.deepcopy(prog_list)
        pnode = prog_list[0]
        for i in range(1,len(prog_list)):
            d_prog = prog_list[i]
            for j in range(len(d_prog.children)):
                if d_prog.children[j].is_param():
                    # replace with pnode
                    d_prog.children[j] = pnode
                    pnode = d_prog
                    continue
        return pnode
    
    '''
    convert a full program to prog_list
    (chain program is assumed)
    '''
    def get_prog_list(self, full_prog):
        def dfs_traverse(dnode):
            # convert to sexp to avoid pointers
            pl = []
            for i in range(len(dnode.children)):
                cnode = dnode.children[i]
                if cnode.is_apply():
                    pl += dfs_traverse(cnode)
                    # change the ApplyNode to ParamNode and for a prog again
                    dnode.children[i] = self.PARAM_NODE
                    break
            pl.append(dnode.to_sexp())
            return pl
        full_prog = copy.deepcopy(full_prog)
        d_sexp_list = dfs_traverse(full_prog)
        d_prog_list = [self.builder._from_sexp(p) for p in d_sexp_list]
        return d_prog_list
        
    '''
    compare the current output with the designated output
    calling the eq function
    '''
    def out_eq(self):
        if len(self.outv_list)==0:
            return False
        return self.eq(self.output, self.outv_list[-1][0]) # access 0 since it's in input format
    
    '''
    return the last output (aka. the current input)
    '''
    def get_frontier(self):
        if len(self.outv_list)==0:
            return self.input
        else:
            return self.outv_list[-1]
        
    '''
    add an sexp, execute and generate intermediate outputs
    if succeeded return True, otherwise return False
    '''
    def add_sexp(self, p_sexp):
        assert len(self.outv_list)==\
               len(self.prog_list)==\
               len(self.sexp_list)
        
        tmp_input = self.get_frontier()
        tmp_prog = self.builder._from_sexp(p_sexp)
        
        try:
            tmp_outv = self.interpreter.eval(tmp_prog,tmp_input)
        except InterpreterError as e:
            # failed to add sexp
            return False
        
        # succeed
        self.prog_list.append(tmp_prog)
        self.sexp_list.append(p_sexp)
        # NOTICE: wrap output in [] to be in the input format
        self.outv_list.append([tmp_outv])
        return True
    
    '''
    make a copy of the current instance, with shared interpreter
    '''
    def make_copy(self):
        new_ps = ProgramSpaceChainOneNB(
            self.spec,
            self.interpreter,
            self.eq,
            self.input,
            self.output,
        )
        new_ps.outv_list = copy.deepcopy(self.outv_list)
        new_ps.prog_list = copy.deepcopy(self.prog_list)
        new_ps.sexp_list = copy.deepcopy(self.sexp_list)
        new_ps.shell_list = copy.deepcopy(self.shell_list)
        new_ps.str_shell_list = copy.deepcopy(self.str_shell_list)
        new_ps.str_shell_dict = copy.deepcopy(self.str_shell_dict)
        return new_ps

In [11]:
def AlphaNeoTrainer(p_config, p_spec, p_interpreter, p_generator, p_model, p_optim, p_writer):
    global DBG_VAR
    reward_list = []
    n_batch = 32
    batch_loss = 0.
    c_nth = 0
    
    total_ac = {1:0,2:0,3:0}
    total_sp = {1:0,2:0,3:0}
    # one program each step
    for d_step in range(p_config["n_steps"]):
        p_model.train()
        
        p_input = p_interpreter.random_table()
        while True:
            p_prog, p_example = p_generator.generate(
                max_depth=p_config["max_depth"],
                example=Example(input=[p_input], output=None),
                probs=(1,5),
            )
            # make sure at least one function call
            if p_prog.is_apply():
                break
                
        # construct the full program
        ps_full = ProgramSpaceChainOneNB(
            p_spec, p_interpreter, eq_r, p_example.input, p_example.output,
        )
        p_prog_list = ps_full.get_prog_list(p_prog)
        for p in p_prog_list:
            ps_full.add_sexp(p.to_sexp())
        total_sp[len(ps_full.prog_list)] += 1
        # start from the first state
        ps_current = ProgramSpaceChainOneNB(
            p_spec, p_interpreter, eq_r, p_example.input, p_example.output,
        )
        d_reward = None
        selected_edges = []
        
        # if true, the problem turns into supervised problem
        if random.random()<=p_config["hint_rate"](d_step,len(ps_full.prog_list)):
            need_hints = True
        else:
            need_hints = False

        # roll till the ending condition
        hc_t = None
        while True:
            outv_current = ps_current.get_frontier()
            # generate dense rep for known nodes
            d_vertex = morpheus_perspective1(outv_current[0])
            # generate dense rep for target
            d_target = morpheus_perspective1(ps_current.output)

            if use_cuda:
                td_vertex = Variable(torch.FloatTensor( [d_vertex] )).cuda()
                td_target = Variable(torch.FloatTensor( [d_target] )).cuda()
            else:
                td_vertex = Variable(torch.FloatTensor( [d_vertex] ))
                td_target = Variable(torch.FloatTensor( [d_target] ))
                                     
            # (1, LIST_PAD_LENGTH+#production_rules)
            td_output, hc_t = p_model(td_vertex, td_target, hc_t)
            cnd_list = ps_full.shell_list

            DBG_VAR = [ps_full, ps_current]
            if need_hints:
                # just assign the correct solution
                selected_id = ps_full.str_shell_dict[
                    str(ps_full.prog_list[len(ps_current.prog_list)])
                ]
            else:
                if random.random()<=p_config["exploration_rate"](d_step):
                    # exploration
                    selected_id = random.choice(range(len(cnd_list)))
                else:
                    # exploitation
                    selected_id = torch.multinomial(td_output.exp().flatten(), 1).cpu().flatten().numpy()[0]
            # keep track of selected edges
            selected_edges.append(td_output[0, selected_id])
            # add selected edges and fill
            ret = ps_current.add_sexp(ps_current.shell_list[selected_id].to_sexp())
            
            if ret==False:
                # failed
                d_reward = 0.
                hc_t = None # cut off sequence
                break
            else:
                # succeeded
                if ps_current.out_eq():
                    # solved in depth less than or equal to max_depth
                    d_reward = 1.
                    total_ac[len(ps_current.prog_list)] += 1
                    hc_t = None # cut off sequence
                    break
                # NOTICE: Trinity depth is # of layers of nodes, AlphaNeo depth is # of edges (height)
                elif len(ps_current.prog_list)>=p_config["max_depth"]-1:
                    d_reward = 0.
                    hc_t = None # cut off sequence
                    break

        # finally compute the loss
        d_loss = 0.
        ns = len(selected_edges)
        for i in range(ns):
            d_decay = p_config["gamma"]**(ns-1-i)
            # should negate the log probabilities
            d_loss += d_decay*d_reward*(-selected_edges[i])

        reward_list.append(d_reward)
        batch_loss += d_loss

        print("\r# STEP{}, size:c{}/f{}, reward:{:.4f}, avg.reward:{:.4f}, ac/sp: 1->{}/{}, 2->{}/{}, 3->{}/{}".format(
            d_step,
            len(ps_current.prog_list), len(ps_full.prog_list), d_reward, sum(reward_list)/len(reward_list),
            total_ac[1], total_sp[1], total_ac[2], total_sp[2], total_ac[3], total_sp[3],
        ), end="")

        if writer is not None:
            writer.add_scalar(
                'avg.reward/step',
                sum(reward_list)/len(reward_list),
                d_step,
            )
            writer.add_scalar(
                'reward/step',
                d_reward,
                d_step,
            )
            writer.add_scalar(
                'sol.length/step',
                ns,
                d_step,
            )
        
        
        c_nth += 1
        if c_nth%n_batch==0:
            c_nth = 0
            # perform gradient in every batch
            batch_loss.backward()
            p_optim.step()
            p_optim.zero_grad()
            batch_loss = 0.
            
        if d_step%10000==0:
            torch.save(p_model.state_dict(), "./saved_models/AlphaNeo_Morpheus_n3.pt".format(d_step))


In [12]:
m_interpreter = MorpheusInterpreter()
m_spec = S.parse_file('./example/mChainOneNB.tyrell')
m_eq = eq_r
m_generator = MorpheusGenerator(
    spec=m_spec,
    interpreter=m_interpreter,
    sfn=m_interpreter.sanity_check,
)
m_ps = ProgramSpaceChainOneNB(
    m_spec, m_interpreter, m_eq, None, None,
)
# m_config = {
#     'n_steps': 1000000,
#     'gamma': 0.618,
#     'exploration_rate': lambda x:0.9-0.8*(min(1, x/10000)),
#     'hint_rate': lambda x,n:{1:1,2:0.5,3:0.8}[n]*(1-(min(0.7, x/10000))),
#     'max_depth': 4,
#     'gen1_length': 27,
# }
m_config = {
    'n_steps': 1000000,
    'gamma': 0.618,
    'exploration_rate': lambda x:0.9-0.8*(min(1, x/10000)),
    'hint_rate': lambda x,n:{1:0.8,2:0.8,3:0.8}[n]*(1-(min(0.7, x/10000))),
    'max_depth': 4,
    'gen1_length': 27,
}
m_config['lstm_input_size'] = 2*m_config["gen1_length"]
m_config["fcs"] = [
    128,
    len(m_ps.shell_list),
]

alpha_neo = AlphaNeo(p_config=m_config)
optimizer = torch.optim.Adam(list(alpha_neo.parameters()))
writer = SummaryWriter("runs/AlphaNeo_Morpheus_n3")
# writer = None

In [13]:
len(m_ps.shell_list)

390

In [ ]:
AlphaNeoTrainer(m_config, m_spec, m_interpreter, m_generator, alpha_neo, optimizer, writer)

# STEP145, size:c1/f3, reward:1.0000, avg.reward:0.8493, ac: 1->38/37, 2->11/6, 3->75/103

In [ ]:
DBG_VAR[0]

In [ ]:
DBG_VAR[0]

In [ ]:
DBG_VAR[1].out_eq()

In [ ]:
DBG_VAR[0].out_eq()

In [ ]:
print(robjects.r(DBG_VAR[0].output))

In [ ]:
print(robjects.r(DBG_VAR[0].outv_list[-1][0]))

In [ ]:
np_obj = numpy.asarray(robjects.r(DBG_VAR[0].outv_list[-1][0])).T

In [ ]:
np_obj.shape

In [ ]:
len(DBG_VAR[0].prog_list)

In [ ]:
len(DBG_VAR[1].prog_list)

In [ ]:
str(DBG_VAR[0].prog_list[len(DBG_VAR[1].prog_list)])